In [1]:
from typing import Optional
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
import numpy as np

In [2]:
username = "icekhan"
version = "latest"

In [3]:
def experiment_op(experiment_name: str,
				  checkpoint_data_size: int,
				  sample_to: int,
				  grid_size: int,
				  n_epochs: int,
				  dataset_root_dir: str,
				  mlflow_backend_host: Optional[str],
				  mlflow_backend_port: Optional[str]):
	
    return dsl.ContainerOp(
        name="experiment-operator",
        image=f"{username}/sapsan_cnn_encoder_experiment:{version}",
        arguments=[
			"--experiment_name", experiment_name,
			"--checkpoint_data_size", checkpoint_data_size,
			"--sample_to", sample_to,
			"--grid_size", grid_size,
			"--n_epochs", n_epochs,
			"--dataset_root_dir", dataset_root_dir,
			"--mlflow_backend_host", mlflow_backend_host,
			"--mlflow_backend_port", mlflow_backend_port
		]
    )


In [4]:
@dsl.pipeline(name="CNN encoder", description="Sapsan CNN encoder experiment")
def pipeline_definition(experiment_name: str = 'CNN encoder experiment',
						checkpoint_data_size: int = 128,
						sample_to: int = 32,
						grid_size: int = 8,
						n_epochs: int = 1,
						dataset_root_dir: str = '/app/dataset/',
						mlflow_backend_host: Optional[str] = None,
						mlflow_backend_port: Optional[str] = None):
	
    experiment_op(experiment_name=experiment_name,
				  dataset_root_dir=dataset_root_dir,
				  checkpoint_data_size=checkpoint_data_size,
				  sample_to=sample_to,
				  grid_size=grid_size,
				  n_epochs=n_epochs,
				  mlflow_backend_host=mlflow_backend_host,
				  mlflow_backend_port=mlflow_backend_port)

In [5]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")


In [6]:
client = kfp.Client()

Failed to load kube config.


In [ ]:
%%bash
tar -xvf pipeline.tar.gz


In [ ]:
run_name = f"Run number {np.random.randint(0, 10000)}"
experiment_name='Sapsan experiment'
# get or create an experiment_id
try:
    experiment_id = client.get_experiment(experiment_name=experiment_name).id
except:
    experiment_id = client.create_experiment(experiment_name).id
experiment_id

In [ ]:
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "n_epochs": 1,
    }
)